In [ ]:
def phase_input_maker(input_fasta, input_filename):
    fasta = open(input_fasta, "r")
    names = []
    sequences = []
    designation = 0
    for line in fasta:
        if designation == 0:
            names.append(line[1:-1])
            designation = 1
        else:
            sequences.append(line[0:-1])
            designation = 0
    fasta.close()
    het_positions = []
    for position in range(len(sequences[0])):
        for sequence in sequences:
            if  sequence[position-1] != 'A' and sequence[position-1] != 'T' and sequence[position-1] != 'C' \
            and sequence[position-1] != 'G' and sequence[position-1] != '-' and sequence[position-1] != '?' \
            and sequence[position-1] != 'B' and sequence[position-1] != 'D' and sequence[position-1] != 'H' \
            and sequence[position-1] != 'V' and sequence[position-1] != 'N' and sequence[position-1] != 'Z':
                if het_positions == []:
                    het_positions.append(position)
                elif position != het_positions[-1]:
                    het_positions.append(position)
    het_IDs = []
    for sequence in sequences:
        het_IDstr = ''
        for position in het_positions:
            het_IDstr += sequence[position-1] + ' '
        het_IDs.append(het_IDstr)
    sepchrom1 = []
    sepchrom2 = []
    for series in het_IDs:
        chromstr1 = ''
        chromstr2 = ''
        for char in series:
            if char == 'A':
                chromstr1 += '0'
                chromstr2 += '0'
            elif char == 'T':
                chromstr1 += '1'
                chromstr2 += '1'
            elif char == 'C':
                chromstr1 += '2'
                chromstr2 += '2'
            elif char == 'G':
                chromstr1 += '3'
                chromstr2 += '3'
            elif char == '-':
                chromstr1 += '-1'
                chromstr2 += '-1'
            elif char == ' ':
                chromstr1 += ' '
                chromstr2 += ' '
            elif char == 'Y':
                chromstr1 += '2'
                chromstr2 += '1'
            elif char == 'R':
                chromstr1 += '0'
                chromstr2 += '3'
            elif char == 'W':
                chromstr1 += '0'
                chromstr2 += '1'
            elif char == 'S':
                chromstr1 += '3'
                chromstr2 += '2'
            elif char == 'K':
                chromstr1 += '1'
                chromstr2 += '3'
            elif char == 'M':
                chromstr1 += '2'
                chromstr2 += '0'
        sepchrom1.append(chromstr1)
        sepchrom2.append(chromstr2)
    input_file = open(input_filename, 'w')
    input_file.write(str(len(sepchrom1)) + '\n')
    input_file.write(str(len(het_positions)) + '\n')
    input_file.write('P ')
    for pos in het_positions:
        input_file.write(str(pos)+' ')
    input_file.write('\n')
    for pos in het_positions:
        input_file.write('M')
    input_file.write('\n')
    for index in range(len(names)):
        input_file.write(str(names[index]) + '\n' + str(sepchrom1[index]) + '\n' + str(sepchrom2[index]) + '\n')
    input_file.close()

In [ ]:
import re

def phase_output_fasta(fasta_file, phase_out_file, phased_fasta1, phased_fasta2):
    fasta = open(fasta_file, "r")
    names = []
    sequences = []
    designation = 0
    for line in fasta:
        if designation == 0:
            names.append(line[1:-1])
            designation = 1
        else:
            sequences.append(line[0:-1])
            designation = 0
    fasta.close()
    phase_out = open(phase_out_file, "r")
    phase_out_text = phase_out.read()
    phase_out.close()
    phase_out_trim = re.search(r'BEGIN BESTPAIRS1[\s\S]*END BESTPAIRS1', phase_out_text).group()
    phase_loop = phase_out_trim[17:-14]
    chrom1 = []
    chrom2 = []
    oot = 0
    for char in phase_loop:
        if oot == 0:
            if char == '\n':
                oot = 1
                charstr = ''
        elif oot == 1:
            if char != '\n':
                if char == '0':
                    charstr += 'A'
                elif char == '1':
                    charstr += 'T'
                elif char == '2':
                    charstr += 'C'
                elif char == '3':
                    charstr += 'G'
            elif char == '\n':
                oot = 2
                chrom1.append(charstr)
                charstr = ''
        elif oot == 2:
            if char != '\n':
                if char == '0':
                    charstr += 'A'
                elif char == '1':
                    charstr += 'T'
                elif char == '2':
                    charstr += 'C'
                elif char == '3':
                    charstr += 'G'
            elif char == '\n':
                oot = 0
                chrom2.append(charstr)
    het_positions = []
    for position in range(len(sequences[0])):
        for sequence in sequences:
            if  sequence[position-1] != 'A' and sequence[position-1] != 'T' and sequence[position-1] != 'C' \
            and sequence[position-1] != 'G' and sequence[position-1] != '-' and sequence[position-1] != '?' \
            and sequence[position-1] != 'B' and sequence[position-1] != 'D' and sequence[position-1] != 'H' \
            and sequence[position-1] != 'V' and sequence[position-1] != 'N' and sequence[position-1] != 'Z':
                if het_positions == []:
                    het_positions.append(position)
                elif position != het_positions[-1]:
                    het_positions.append(position)
    fasta1 = ''
    fasta2 = ''
    for ind1 in range(len(names)):
        fasta1 += '>' + names[ind1] + '\n'
        fasta2 += '>' + names[ind1] + '\n'
        het_pos_ticker = 0
        for ind2 in range(len(sequences[ind1])):
            if ind2 != het_positions[het_pos_ticker]-1:
                fasta1 += sequences[ind1][ind2]
                fasta2 += sequences[ind1][ind2]
            elif ind2 == het_positions[het_pos_ticker]-1 and het_pos_ticker < len(het_positions)-1:
                fasta1 += chrom1[ind1][het_pos_ticker]
                fasta2 += chrom2[ind1][het_pos_ticker]
                het_pos_ticker += 1
            elif ind2 == het_positions[het_pos_ticker]-1:
                fasta1 += chrom1[ind1][het_pos_ticker]
                fasta2 += chrom2[ind1][het_pos_ticker]
        fasta1 += '\n'
        fasta2 += '\n'
    fasta1_file = open(phased_fasta1, 'w')
    fasta1_file.write(fasta1)
    fasta1_file.close()
    fasta2_file = open(phased_fasta2, 'w')
    fasta2_file.write(fasta2)
    fasta2_file.close()